<a href="https://colab.research.google.com/github/whoami-Lory271/DL-project/blob/main/TPTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install library import 

In [1]:
!pip install pytorch-lightning --quiet
!pip install torchmetrics --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 22.4 MB/s eta 0:00:00


In [1]:
from numpy import array
from sklearn.preprocessing import OneHotEncoder
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.normalization import LayerNorm

from numpy import array
from sklearn.preprocessing import OneHotEncoder
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.normalization import LayerNorm
import pickle
import os
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from pathlib import Path
import pandas as pd
import numpy as np
import csv
import torchmetrics
import pytorch_lightning as pl
from pytorch_lightning.callbacks.progress import TQDMProgressBar

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Helper Function

In [2]:
def get_padding_mask(X):
  pad = X == 0
  padding_mask = pad.repeat(1,1,X.shape[1]).reshape((X.shape[0],X.shape[1],X.shape[1]))
  padding_mask[pad] = True
  return padding_mask

In [3]:
def create_padding(encode, vocab, max_len):
    enc = np.array(encode)
    encoded = np.zeros(max_len + 2)
    for k in range(len(enc)):
      encoded[k] = enc[k]
    return encoded

#Preprocessing

In [4]:
save_path = "/content/drive/MyDrive/Deep learning" #change if needed

Path for easy train data:

In [5]:
#take all the txt files
easy_path = os.path.join(save_path,"easy_pickle")
easy_texts_path=[str(path) for path in Path(easy_path).glob("*.txt")]
print(easy_texts_path)

['/content/drive/MyDrive/Deep learning/easy_pickle/algebra__linear_1d_easy.txt', '/content/drive/MyDrive/Deep learning/easy_pickle/calculus__differentiate_easy.txt', '/content/drive/MyDrive/Deep learning/easy_pickle/probability__swr_p_level_set_easy.txt']


Path for medium train data:

In [ ]:
medium_path= os.path.join(save_path,"medium_pickle")
medium_texts_path=[str(path) for path in Path(medium_path).glob("*.txt")]
print(medium_texts_path)

['/content/drive/MyDrive/Deep learning/medium_pickle/algebra__linear_1d_medium.txt', '/content/drive/MyDrive/Deep learning/medium_pickle/calculus__differentiate_medium.txt', '/content/drive/MyDrive/Deep learning/medium_pickle/probability__swr_p_level_set_medium.txt']


Path for hard train data:

In [ ]:
hard_path= os.path.join(save_path,"hard_pickle")
hard_texts_path=[str(path) for path in Path(hard_path).glob("*.txt")]
print(hard_texts_path)

['/content/drive/MyDrive/Deep learning/hard_pickle/algebra__linear_1d_hard.txt', '/content/drive/MyDrive/Deep learning/hard_pickle/calculus__differentiate_hard.txt', '/content/drive/MyDrive/Deep learning/hard_pickle/probability__swr_p_level_set_hard.txt']


Path for interpolate data:

In [6]:
interpolate_path= os.path.join(save_path,"interpolate_pickle")
interpolate_texts_path=[str(path) for path in Path(interpolate_path).glob("*.txt")]
print(interpolate_texts_path)

['/content/drive/MyDrive/Deep learning/interpolate_pickle/algebra__linear_1d.txt', '/content/drive/MyDrive/Deep learning/interpolate_pickle/calculus__differentiate.txt', '/content/drive/MyDrive/Deep learning/interpolate_pickle/probability__swr_p_level_set.txt']


In [ ]:
def create_vocabulary_from_directory(sentences_path):
  vocabulary = {}
  vocabulary['<sos>'] = 1
  vocabulary['<eos>'] = 2
  vocabulary['<pad>'] = 0
  index = 3
  conta = 0
  for file in sentences_path:
    with open(file, 'r') as file_in:
      text = file_in.read()
      sentences = text.split('\n')
      sentences_new = sentences[:-1] #remove last element because it's empty
      print("num sentences", len(sentences_new))
      for s in sentences_new:
        conta +=1
        for t in s:
         if t not in vocabulary:
            vocabulary[t] = index
            index += 1
      file_in.close()
  return vocabulary, conta

In [ ]:
# vocabulary, conta = create_vocabulary_from_directory(ft_texts_path)
# print(conta)

In [ ]:
# #create the pickle
# outfile = open(os.path.join(save_path,"vocabs"), 'wb')
# pickle.dump(vocabulary, outfile)
# outfile.close()

In [ ]:
def encoding_from_sentence(vocab, sentence):
  enc= [1]
  for token in sentence:
    enc.append(vocab[token])
  enc.append(2)
  return enc

In [ ]:
def create_pickles(vocabulary, sentences_paths, column_names=["Encoding"], path_quest="/content/drive/MyDrive/Deep learning/questions", path_answ="/content/drive/MyDrive/Deep learning/answers"):
  quest = []
  answ = []
  out_file_quest = open(path_quest, 'wb')
  out_file_answ = open(path_answ, 'wb')
  for file in sentences_paths:
    with open(file, 'r') as file_in:
      text = file_in.read()
      sentences = text.split('\n')
      sentences = sentences[:-1]
      print("num sentences", len(sentences))
      for i, sentence in enumerate(sentences):
          enc = encoding_from_sentence(vocabulary, sentence)
          if i % 2 == 0: #question
            #quest.append({"Sentence": sentence, "Encoding": enc})
            quest.append({"Encoding": enc})
          elif i % 2 == 1: #answer
            #answ.append({"Sentence": sentence, "Encoding": enc})
            answ.append({"Encoding": enc})
  pickle.dump(quest, out_file_quest)
  pickle.dump(answ, out_file_answ)
  out_file_quest.close()
  out_file_answ.close()
  return path_quest, path_answ

##Dataset

In [7]:
#unpickle the vocabulary
infile = open(os.path.join(save_path,'vocabs'),'rb')
vocab = pickle.load(infile)
infile.close()

Execute if you want to train easy data:

In [8]:
#unpickle the qestions
with open(os.path.join(easy_path,"questions"),'rb') as infile:
  quest = pickle.load(infile)
print(len(quest))

#unpickle the answers
with open(os.path.join(easy_path,"answers"),'rb') as infile:
  answ = pickle.load(infile)
print(len(answ))

1999998
1999998


Execute if you want to train medium data:

In [9]:
#unpickle the qestions
with open(os.path.join(medium_path,"questions"),'rb') as infile:
  quest = pickle.load(infile)
print(len(quest))

#unpickle the answers
with open(os.path.join(medium_path,"answers"),'rb') as infile:
  answ = pickle.load(infile)
print(len(answ))

NameError: ignored

Execute if you want to train hard data:

In [ ]:
#unpickle the qestions
with open(os.path.join(hard_path,"questions"),'rb') as infile:
  quest = pickle.load(infile)
print(len(quest))

#unpickle the answers
with open(os.path.join(hard_path,"answers"),'rb') as infile:
  answ = pickle.load(infile)
print(len(answ))

Execute to load test:

In [10]:
#unpickle the qestions
with open(os.path.join(interpolate_path,"questions"),'rb') as infile:
  test_quest = pickle.load(infile)
print(len(test_quest))

#unpickle the answers
with open(os.path.join(interpolate_path,"answers"),'rb') as infile:
  test_answ = pickle.load(infile)
print(len(test_answ))

30000
30000


In [11]:
class MathematicsDataset(Dataset):
  def __init__(self, quest, answ, vocab , max_len_quest=160, max_len_answ=30):
    self.quest = quest
    self.answ = answ
    self.vocab = vocab
    self.max_len_quest = max_len_quest
    self.max_len_answ = max_len_answ

  def __len__(self):
    return len(self.quest)

  def __getitem__(self, idx):
    assert(idx  < len(self.quest)) #indices should start from 0 to len - 1 (there are 5999994 elements)
    question = self.quest[idx]["Encoding"]
    encoding1 = torch.from_numpy(create_padding(question, self.vocab, self.max_len_quest))
    encoding1 = encoding1.type(torch.int64)
    #self.sample["Question"] = encoding
      
    answer = self.answ[idx]["Encoding"]
    encoding2 = torch.from_numpy(create_padding(answer, self.vocab, self.max_len_answ))
    encoding2 = encoding2.type(torch.int64)
    #self.sample["Answer"] = encoding
      
    return encoding1, encoding2 

In [12]:
class PlMathematicsDataset(pl.LightningDataModule):
    def __init__(self, train_quest, train_answ, vocab , test_quest, test_answ, max_len_quest=160, max_len_answ=30, batch_size=128):
        super().__init__()
        self.train_quest = train_quest
        self.train_answ = train_answ
        self.test_quest = test_quest
        self.test_answ = test_answ
        self.vocab = vocab
        self.max_len_quest = max_len_quest
        self.max_len_answ = max_len_answ
        self.batch_size = batch_size
        self.dataset = MathematicsDataset(self.train_quest, self.train_answ, self.vocab)

    # def prepare_data(self):
        # tok, load, ecc...
        # MNIST(self.data_dir, train=True, download=True)
        # MNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):

        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            self.train_ds = MathematicsDataset(self.train_quest, self.train_answ, self.vocab)

        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
            self.test_ds = MathematicsDataset(self.test_quest, self.test_answ, self.vocab)

        if stage == "predict" or stage is None:
            self.predict_ds = MathematicsDataset(self.test_quest, self.test_answ, self.vocab)

    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.batch_size, shuffle = True)

    def test_dataloader(self):
        return DataLoader(self.test_ds, batch_size=self.batch_size)

    def predict_dataloader(self):
        return DataLoader(self.predict_ds, batch_size=self.batch_size)

In [13]:
#Create dataset
dataset = PlMathematicsDataset(quest, answ, vocab, test_quest, test_answ)

In [ ]:
# #Create dataloader
# train_loader = DataLoader(dataset, 32, shuffle=True)
# print(len(train_loader))

In [ ]:
# print("data", next(iter(train_loader)))
# print("questions_shape ", next(iter(train_loader))[0].shape)

# print("answers_shape ", next(iter(train_loader))[1].shape)

# Model

In [14]:
class EmbeddingLayer(nn.Module):
  def __init__(self,vocabulary_size,embedding_dim):
    super().__init__()
    self.E = nn.Embedding(vocabulary_size,embedding_dim)
  
  def forward(self,x):
    return self.E(x)

In [15]:
def position_embedding(batch_size,seq_length,emb_dim,pad_mask):
  res = torch.zeros((batch_size,seq_length,emb_dim),dtype=torch.float32)
  for pos in range(seq_length):
    for i in range(emb_dim):
      if i%2 == 0:
        res[:,pos,i] = math.sin(pos/10000**(2*i/emb_dim))
      else:
        res[:,pos,i] = math.cos(pos/10000**(2*i/emb_dim))
  res[pad_mask[:,0,:]] = 0
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  return res.to(device)

In [16]:
class TPSelfAttention(nn.Module):
  def __init__(self,dz,dk):
    super().__init__()
    self.dk = dk
    self.Wq = nn.Linear(dz,dk)
    self.Wk = nn.Linear(dz,dk)
    self.Wv = nn.Linear(dz,dk)
    self.Wr = nn.Linear(dz,dk)
    self.Wo = nn.Linear(dk,dz)
    self.softmax = nn.Softmax(dim=2)
  
  def forward(self,x,padding_mask,enc=None,mask=False,other_mask=None):
    q = self.Wq(x)
    r = self.Wr(x)
    if enc == None:
      k = self.Wk(x)
      v = self.Wv(x)
    else:
      k = self.Wk(enc)
      v = self.Wv(enc)
    sc = torch.matmul(q,k.permute(0,2,1)) / math.sqrt(self.dk)

    if other_mask == None:
      sc[padding_mask] = float('-inf')
    else:
      qmod = q.clone()
      kmod = k.clone()
      qmod[padding_mask[:,0,:]] = 0
      kmod[other_mask[:,0,:]] = 0
      sc = torch.matmul(qmod,kmod.permute(0,2,1)) / math.sqrt(self.dk)
      sc[sc == 0] = float('-inf')

    if mask==True:
      for i in range(sc.shape[1]):
        sc[:,i,i+1:] = float('-inf')
    score = torch.matmul(torch.nan_to_num(self.softmax(sc)),v)
    return self.Wo(r * score)

In [17]:
class TPMultiHeadAttention(nn.Module):
  def __init__(self,dmodel,nhead,dropout=0.1):
    super().__init__()
    self.nhead = nhead
    self.att_layers = nn.ModuleList([TPSelfAttention(dmodel,dmodel // nhead) for i in range(nhead)])
    self.drop = nn.Dropout(p=dropout)
  
  def forward(self,x,padding_mask,enc=None,mask=False,other_mask=None):
    y = self.att_layers[0](x,padding_mask,enc=enc,mask=mask,other_mask=other_mask)
    for i in range(1,self.nhead):
      y += self.att_layers[i](x,padding_mask,enc=enc,mask=mask,other_mask=other_mask)
    y = self.drop(y)
    return y

In [18]:
class FFN(nn.Module):
  def __init__(self,dmodel,df,dropout=0.1):
    super().__init__()
    self.W1 = nn.Linear(dmodel,df)
    self.W2 = nn.Linear(df,dmodel)
    self.drop = nn.Dropout(p=dropout)
  
  def forward(self,x):
    x = self.W1(x)
    x = F.relu(x)
    x = self.W2(x)
    x = self.drop(x)
    return x

In [19]:
class Encoder(nn.Module):
  def __init__(self,dmodel,df,nhead):
    super().__init__()
    self.norm = LayerNorm(dmodel)
    self.mha = TPMultiHeadAttention(dmodel,nhead)
    self.norm1 = LayerNorm(dmodel)
    self.ffn = FFN(dmodel,df)
    self.norm2 = LayerNorm(dmodel)
  
  def forward(self,x,padding_mask):
    x = self.norm(x)
    z = self.mha(x,padding_mask)
    z = self.norm1(x+z)
    y = self.ffn(z)
    return self.norm2(z+y)

In [20]:
class Decoder(nn.Module):
  def __init__(self,dmodel,df,nhead):
    super().__init__()
    self.norm = LayerNorm(dmodel)
    self.masked_mha = TPMultiHeadAttention(dmodel,nhead)
    self.norm1 = LayerNorm(dmodel)
    self.enc_dec_attention = TPMultiHeadAttention(dmodel,nhead)
    self.norm2 = LayerNorm(dmodel)
    self.ffn = FFN(dmodel,df)
    self.norm3 = LayerNorm(dmodel)
  
  def forward(self,x,enc,padding_mask,other_mask):
    x = self.norm(x)
    z1= self.masked_mha(x,padding_mask,mask=True)
    z1 = self.norm1(x+z1)
    z2= self.enc_dec_attention(z1,padding_mask,enc=enc,mask=False,other_mask=other_mask)
    z2 = self.norm2(z1+z2)
    y = self.ffn(z2)
    return self.norm3(z2+y)

In [21]:
class TPTransformer(pl.LightningModule):
    def __init__(self,voc_size,dmodel,df,nhead,nlayers,dropout=0.1):
      super().__init__()
      self.save_hyperparameters()
      self.nlayers = nlayers
      self.embedding = EmbeddingLayer(voc_size,dmodel)
      self.encoders = nn.ModuleList([Encoder(dmodel,df,nhead) for i in range(nlayers)])
      self.decoders = nn.ModuleList([Decoder(dmodel,df,nhead) for i in range(nlayers)])
      self.drop1 = nn.Dropout(p=dropout)
      self.drop2 = nn.Dropout(p=dropout)
      self.dmodel = dmodel
      self.metric = nn.CrossEntropyLoss()
      self.test_acc = torchmetrics.Accuracy(task='multiclass', num_classes = voc_size)

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        input, teacher = batch
        in_pad_mask = get_padding_mask(input)
        out_pad_mask = get_padding_mask(teacher)
        x,output = tran(input,teacher,in_pad_mask,out_pad_mask)
        pred = output[:,:-1,:].reshape(output.shape[0]*(output.shape[1] - 1),output.shape[2])
        target = teacher[:,1:].reshape(teacher.shape[0]*(teacher.shape[1] - 1))
        loss = self.metric(pred,target)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(tran.parameters(),lr=1e-04, betas=(0.9, 0.995), eps=1e-09)
        return optimizer

    def test_step(self, batch, batch_idx):
        x,y = batch
        enc = True
        trg = y.clone()
        input_pad_mask = get_padding_mask(x)
        for i in range(1,y.shape[1]):
          output_pad_mask = get_padding_mask(trg[:,:i])
          x,output = tran(x,trg[:,:i],input_pad_mask,output_pad_mask,encoding = enc)
          trg[:,i] = output.argmax(-1)[:,-1]
          enc = False
        target_pad_mask = get_padding_mask(y[:,1:])
        acc = self.test_acc(output.argmax(-1)[target_pad_mask[:,0,:]], y[:,1:][target_pad_mask[:,0,:]])
        self.log('test_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return acc

    def forward(self,x,z,in_padding_mask,out_padding_mask,encoding=True):
      if encoding:
        emb = self.embedding(x)
        t = position_embedding(x.shape[0],x.shape[1],self.dmodel,in_padding_mask)
        x = emb + t
        x = self.drop1(x)
        for i in range(self.nlayers):
          x = self.encoders[i](x,in_padding_mask)

      emb = self.embedding(z)
      t = position_embedding(z.shape[0],z.shape[1],self.dmodel,out_padding_mask)
      z = emb + t
      z = self.drop2(z)
      for i in range(self.nlayers):
        z = self.decoders[i](z,x,out_padding_mask,in_padding_mask)
      
      z = z @ self.embedding.E.weight.T
      z[out_padding_mask[:,0,:]] = 0

      return x,z

    def predict_step(self, batch, batch_idx):
        x,y = batch
        enc = True
        trg = y.clone()
        input_pad_mask = get_padding_mask(x)
        for i in range(1,y.shape[1]):
          output_pad_mask = get_padding_mask(trg[:,:i])
          x,output = tran(x,trg[:,:i],input_pad_mask,output_pad_mask,encoding = enc)
          trg[:,i] = output.argmax(-1)[:,-1]
          enc = False
        return output.argmax(-1)

In [22]:
dmodel = 512
dk,dv = 64,64
nhead = 8
df = 512
nlayers = 6
vocabulary_size = len(vocab)
enable_checkpointing = True
resume_training = False
load_model = False
load_model_dir = "/content/drive/MyDrive/Deep learning/model_checkpoints/TPTransformer/last.ckpt" #change path to load a different checkpoint

In [23]:
tran = TPTransformer(vocabulary_size,dmodel,df,nhead,nlayers)
if load_model == True:
  tran = TPTransformer.load_from_checkpoint(load_model_dir)

In [24]:
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(dirpath="/content/drive/MyDrive/Deep learning/model_checkpoints/TPTransformer", save_last =True, save_on_train_epoch_end = True)
# define the logger object 
logger = TensorBoardLogger("tb_logs", name="TPTransformer",log_graph=True )

#build the correct callbacks
callbacks = [TQDMProgressBar(refresh_rate=20)]
ckpt_path = load_model_dir

#handle checkpointing
if resume_training == False:
  ckpt_path = None
if enable_checkpointing == True:
  callbacks = [checkpoint_callback, TQDMProgressBar(refresh_rate=20)]
  
#passing it to the trainer
gpu = 0
if torch.cuda.is_available() : 
  gpu = 1
trainer = pl.Trainer(enable_checkpointing=enable_checkpointing ,gpus=gpu, max_epochs=3, logger=logger, callbacks=callbacks)
trainer.fit(ckpt_path=ckpt_path, model=tran, datamodule=dataset)
print(checkpoint_callback.best_model_path)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:467: LightningDeprecationWarning: Setting `Trainer(gpus=0)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=0)` instead.
  rank_zero_deprecation(
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type               | Params
-------------------------------------------------
0 | embedding | EmbeddingLayer     | 29.7 K
1 | encoders  | ModuleList         | 11.1 M
2 | decoder

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


/content/drive/.shortcut-targets-by-id/1lKplZjRr839VnzCbCDMjd--WHVhTwnbR/Deep learning/model_checkpoints/TPTransformer/epoch=2-step=12.ckpt


Execute if you want interpolate data:

In [25]:
trainer.test(tran,datamodule=dataset)

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_acc_epoch                0.875
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc_epoch': 0.875}]